In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib qt

In [ ]:
data_path = ''

eog = [61, 62, 63]

data = mne.io.read_raw_gdf(data_path, preload=True, eog=eog)

data.drop_channels(data.ch_names[64:])

bads = 11, 24

In [3]:
data.info

<Info | 7 non-empty values
 bads: []
 ch_names: eeg-0, eeg-1, eeg-2, eeg-3, eeg-4, eeg-5, eeg-6, eeg-7, eeg-8, ...
 chs: 61 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: 2015-05-26 15:42:48 UTC
 nchan: 64
 projs: []
 sfreq: 512.0 Hz
>

In [4]:
data.plot(scalings='auto', n_channels=40, duration=20);

### Removing Bad Channels

In [5]:
data = data.crop(tmin=5)

In [6]:
eog_preprocessing, _ = mne.preprocessing.compute_proj_eog(data, n_grad=1, n_mag=1, n_eeg=1, no_proj=True, reject=None)

Running EOG SSP computation
EOG channel index for this subject is: [61 62 63]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 8192 samples (16.000 sec)

Now detecting blinks and generating corresponding events
No significant peaks found
Number of EOG events detected : 0


C:\ProgramData\Anaconda3\envs\mne\lib\site-packages\mne\preprocessing\_peak_finder.py:67: RuntimeWarning: invalid value encountered in less
  ind = np.where(dx0[:-1:] * dx0[1::] < 0)[0] + 1
<ipython-input-6-bc1df270c765>:1: RuntimeWarning: No EOG events found, returning None for projs
  eog_preprocessing, _ = mne.preprocessing.compute_proj_eog(data1, n_grad=1, n_mag=1, n_eeg=1, no_proj=True, reject=None)


### Creating epochs

In [7]:
annotations = data.annotations

event_id = {'1536': 1, '1537': 2, '1538': 3, '1539': 4, '1540': 5, '1541': 6, '1542': 7}
events, events_dict = mne.events_from_annotations(data, event_id=event_id)

Used Annotations descriptions: ['1536', '1537', '1538', '1539', '1540', '1541', '1542']


In [8]:
events_dict_corrected = {'elow/flexion': 1, 'elbow/extension': 2, 'arm/supination': 3,
              'arm/pronation': 4, 'hand/close': 5, 'hand/open': 6, 'rest': 7}

fig = mne.viz.plot_events(events, event_id=events_dict_corrected, sfreq=data.info['sfreq'],
                          first_samp=data.first_samp);

In [9]:
epochs = mne.Epochs(raw=data, events=events, event_id=events_dict_corrected, tmin=-0.5, tmax=3, preload=True)

42 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 42 events and 1793 original time points ...
0 bad epochs dropped


In [10]:
epochs.__len__

<bound method GetEpochsMixin.__len__ of <Epochs  |   42 events (all good), -0.5 - 3 sec, baseline [None, 0], ~36.9 MB, data loaded,
 'arm/pronation': 6
 'arm/supination': 6
 'elbow/extension': 6
 'elow/flexion': 6
 'hand/close': 6
 'hand/open': 6
 'rest': 6>>

In [11]:
epochs['arm/pronation'].plot(scalings='auto', n_channels=10);

Dropped 0 epochs: 
Channels marked as bad: []


### Filtering

In [12]:
data_filtered = epochs.filter(l_freq=50, h_freq=0.5, method='fft')

Setting up band-stop filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3379 samples (6.600 sec)



<ipython-input-12-5ebb3384d33f>:1: RuntimeWarning: filter_length (3379) is longer than the signal (1793), distortion is likely. Reduce filter length or filter a longer signal.
  data_filtered = epochs.filter(l_freq=50, h_freq=0.5, method='fft')


In [13]:
data_filtered.plot(scalings='auto', events=events);

Dropped 0 epochs: 
Channels marked as bad: []


### Downsampling

In [14]:
print('Original sampling rate: ', epochs.info['sfreq'], 'Hz')
epochs_resampled = epochs.copy().resample(256, npad='auto')
print('New sampling rate: ', epochs_resampled.info['sfreq'], 'Hz')

Original sampling rate:  512.0 Hz
New sampling rate:  256.0 Hz


In [15]:
plt.figure(figsize=(10, 6))

n_samples = int(0.5 * epochs.info['sfreq'])
plt.plot(epochs.times[:n_samples], epochs.get_data()[0, 0, :n_samples], color='black')

n_samples = int(0.5 * epochs_resampled.info['sfreq'])
plt.plot(epochs_resampled.times[:n_samples], epochs.get_data()[0, 0, :n_samples], color='red')

plt.xlabel('time [s]')
plt.legend(['original at 512Hz', 'sampled to 256Hz'], loc='best')
plt.title('Original vs downsamples epochs')
mne.viz.tight_layout()

### Saving to DataFrame

In [16]:
epochs_resampled.drop_channels(['eeg-12', 'eeg-17', 'eeg-0', 'eeg-19'])

<Epochs  |   42 events (all good), -0.5 - 2.99609 sec, baseline [None, 0], ~17.3 MB, data loaded,
 'arm/pronation': 6
 'arm/supination': 6
 'elbow/extension': 6
 'elow/flexion': 6
 'hand/close': 6
 'hand/open': 6
 'rest': 6>

In [17]:
df = epochs_resampled.to_data_frame()
df.drop(['time', 'condition', 'epoch'], axis=1, inplace=True)
df.head()

,eeg-1,eeg-2,eeg-3,eeg-4,eeg-5,eeg-6,eeg-7,eeg-8,eeg-9,eeg-10,...,eeg-54,eeg-55,eeg-56,eeg-57,eeg-58,eeg-59,eeg-60,eog-r,eog-m,eog-l
0,1.233366e+07,7.917713e+06,9.092649e+06,1.126667e+07,3.198847e+06,5.360397e+06,9.002609e+06,8.571124e+06,5.239841e+06,4.295213e+06,...,2.746856e+06,4.280375e+06,4.177384e+06,1.549734e+06,4.384880e+05,2.678838e+06,2.199416e+06,1.347109e+07,7.225095e+07,1.712420e+07
1,1.986016e+07,1.477457e+07,1.551380e+07,1.214882e+07,1.036077e+07,1.747457e+07,1.671733e+07,1.533228e+07,1.302706e+07,1.288625e+07,...,1.068404e+07,1.002737e+07,9.233302e+06,5.889657e+06,8.034194e+06,6.695028e+06,7.683814e+06,2.253770e+07,7.607093e+07,1.670947e+07
2,1.103461e+07,4.581639e+06,5.743556e+06,1.087542e+06,2.523630e+06,7.098404e+06,7.524738e+06,6.760655e+06,5.871404e+06,6.088265e+06,...,-3.167736e+06,-1.321504e+06,-5.054550e+04,-4.196344e+06,-9.790520e+06,3.688024e+05,-1.520672e+06,1.588090e+07,6.447120e+07,1.293665e+07
3,1.640156e+07,1.105414e+07,1.148467e+07,1.016231e+07,9.887551e+06,1.128725e+07,1.359890e+07,1.149927e+07,8.395449e+06,8.138911e+06,...,4.841418e+06,7.223950e+06,8.582398e+06,6.190034e+06,4.533077e+06,7.330882e+06,7.640946e+06,1.484683e+07,6.275665e+07,2.091764e+07
4,7.718311e+06,1.786803e+06,2.930772e+06,2.791440e+06,-7.970332e+06,8.058564e+05,3.314433e+06,2.963867e+06,1.660303e+06,4.467629e+05,...,4.416594e+05,3.704836e+05,-2.158884e+06,-5.355998e+06,-7.541921e+06,-2.839770e+06,-4.068110e+06,-7.728377e+05,4.657414e+07,6.191810e+06


1 epochs: 896

In [18]:
df.to_csv(path, index=False)